**Table of contents**<a id='toc0_'></a>    
- [Vector Stores and Retrievers in LangChain](#toc1_)    
  - [Documents](#toc1_1_)    
  - [Retrievers in LangChain](#toc1_2_)    
  - [RAG Chain](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Vector Stores and Retrievers in LangChain](#toc0_)

LangChain helps computers understand and find information quickly using special tools called vector stores and retrievers. These tools are like smart libraries that help language models find the right information fast.

What You'll Learn:
- What documents are
- How vector stores save information 
- How retrievers find exactly what you need

Main Goals:
- Help AI get the right information
- Make search results more accurate
- Allow computers to understand context better

Think of it like a super-smart search engine that understands not just words, but the meaning behind them. Instead of just matching keywords, these tools can find information that really matches what you're looking for.

We'll explore how these cool technologies work together to make AI smarter and more helpful.

In [ ]:
%pip install langchain
%pip install langchain-chroma
%pip install langchain_groq

## <a id='toc1_1_'></a>[Documents](#toc0_)
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

- API Key and Environment Setup: Configures environment variables and initializes the Groq language model using Llama3-8b.
- Model Initialization: Prepares the ChatGroq model for interaction using API credentials.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Set env
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

# Set the model
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B791236BD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B7912375F0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
%pip install langchain_huggingface

- Embedding Model Setup: Configures an open-source embedding model from Hugging Face for converting text into vector representations.
- Vectorization Preparation: Initializes the "all-MiniLM-L6-v2" model for efficient text embedding

In [ ]:
# Import required libraries for Hugging Face embedding
import tqdm as notebook_tqdm
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize a lightweight, open-source HuggingFace embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

- Chroma Vector Store Initialization: Creates a vector database using ChromaDB with the previously configured HuggingFace embeddings.
- Document Vectorization: Converts documents into a searchable vector store for efficient retrieval.

In [11]:
# Import Chroma vector store from LangChain
from langchain_chroma import Chroma

# Create a vector store by embedding the documents using the Hugging Face embeddings
vectorstore = Chroma.from_documents(documents, embedding=embeddings)

# Display the vector store configuration
vectorstore

In [12]:
# Conduct a semantic similarity search for "cat"
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

- Asynchronous Semantic Search: Performs an asynchronous similarity search for "cat" in the vector store.
- Non-Blocking Retrieval: Enables concurrent document retrieval without blocking the main execution thread.

In [13]:
# asynchronous semantic similarity search for "cat"

await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [ ]:
# similarity search with relevance scoring for "cat"
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740896463394165),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740896463394165)]

## <a id='toc1_2_'></a>[Retrievers in LangChain](#toc0_)

LangChain VectorStore objects cannot be directly integrated into Expression Language chains because they do not inherit from the Runnable class.

LangChain Retrievers are Runnables that implement standard methods (synchronous, asynchronous, and batch operations) and are designed for LCEL chain integration.

You can create a simple Runnable without subclassing Retriever by choosing a document retrieval method like similarity_search.

Retrievers are preferred in LangChain because they:
1. Directly integrate into LCEL chains
2. Support flexible retrieval methods
3. Enable seamless chained operations

This approach provides an easy, standardized way to retrieve and use documents in AI applications.

- Custom Retriever Creation: Builds a custom Runnable retriever using RunnableLambda with the vectorstore's similarity_search method.
- Batch Retrieval: Demonstrates retrieving documents for multiple queries simultaneously.

    - Uses RunnableLambda to wrap vectorstore's similarity_searc
    - Sets a fixed number of results per query (k=1)
    - Enables batch retrieval for multiple search terms
    - Converts vectorstore into a Runnable for LCEL integration

In [15]:
# Import necessary type hints and LangChain classes
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Create a custom retriever with a fixed number of results (k=1)
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

# Perform batch retrieval for multiple queries
retriever.batch(["cat", "dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

- Vectorstores: Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Use similarity-based search
    search_kwargs={"k": 1}     # Limit to 1 result per query
)

# Perform batch retrieval for multiple queries
retriever.batch(["cat", "dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

## <a id='toc1_3_'></a>[RAG Chain](#toc0_)

- RAG Chain Setup: Creates a Retrieval-Augmented Generation (RAG) workflow that combines document retrieval with language model response generation.
- Context-Aware Question Answering: Builds a system that uses retrieved documents to inform the model's answer.

In [ ]:
## RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define a template for context-based answering
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
# Create a prompt template from the message
prompt = ChatPromptTemplate.from_messages([("human", message)])

# Construct the RAG chain
rag_chain = {
    "context": retriever,        # Retrieve relevant documents
    "question": RunnablePassthrough()  # Pass the original question
} | prompt | llm  # Add prompt template and language model

# Invoke the chain with a question
response = rag_chain.invoke("tell me about dogs")

# Print the model's response
print(response.content)


According to the provided context, dogs are great companions, known for their loyalty and friendliness.


1. **`message`**
   - Defines a message template to generate a response to the user's question using only the provided context.

2. **`ChatPromptTemplate.from_messages()`**
   - Converts the message template into a **ChatPromptTemplate** object.
   - **`{question}`** and **`{context}`** placeholders are dynamically filled.

3. **`RunnablePassthrough()`**
   - **Purpose:** Passes the input through without processing.
   - Here, it directly transfers the value corresponding to the `"question"` key to the chain.

4. **`rag_chain`**
   - **Chaining Process:**
     - **`"context": retriever`**: Retriever obtains the context and provides necessary information for the response.
     - **`"question": RunnablePassthrough()`**: The question is added to the chain without modification.
     - **`| prompt | llm`**: Response is generated from the model based on context and question.

5. **`rag_chain.invoke("tell me about dogs")`**
   - **Input:** `"tell me about dogs"`.
   - Chain:
     - **Retriever:** Fetches context relevant to "dogs" query.
     - **Prompt:** Transforms question and context into model input.
     - **LLM:** Produces response.

6. **`response.content`**
   - **Purpose:** Returns the text content of the response generated by the model.

**Result:** A response is generated using only the context for the user's "tell me about dogs" question.

In [ ]:
# THE END